# Capstone project predicting insurance risk analysis

In [3]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [4]:
#import data files: files in CSV format
df =pd.read_csv('train.csv', index_col='Id')



In [5]:
#Check if there are any missing data with count() method and fill it with df.fillna()
df.shape
df.count()
#df = df.fillna(0)

Product_Info_1         59381
Product_Info_2         59381
Product_Info_3         59381
Product_Info_4         59381
Product_Info_5         59381
Product_Info_6         59381
Product_Info_7         59381
Ins_Age                59381
Ht                     59381
Wt                     59381
BMI                    59381
Employment_Info_1      59362
Employment_Info_2      59381
Employment_Info_3      59381
Employment_Info_4      52602
Employment_Info_5      59381
Employment_Info_6      48527
InsuredInfo_1          59381
InsuredInfo_2          59381
InsuredInfo_3          59381
InsuredInfo_4          59381
InsuredInfo_5          59381
InsuredInfo_6          59381
InsuredInfo_7          59381
Insurance_History_1    59381
Insurance_History_2    59381
Insurance_History_3    59381
Insurance_History_4    59381
Insurance_History_5    33985
Insurance_History_7    59381
                       ...  
Medical_Keyword_20     59381
Medical_Keyword_21     59381
Medical_Keyword_22     59381
Medical_Keywor

In [6]:
# Datatypes of the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59381 entries, 2 to 79146
Columns: 127 entries, Product_Info_1 to Response
dtypes: float64(18), int64(108), object(1)
memory usage: 58.0+ MB


Summary Statistcis for all the variables to check for Mean, SD, 25th quantile, 75th quantile, Min, Max 

In [ ]:
#Summary Statistics of the Data Frame
df.describe()

Univariate Analysis to check for dependent variable. The dependent varaible has eight discrete values. Use bar plot to compare the eight distinct responses.

In [ ]:
# Univariate analysis of the Response data - bar plot
df.groupby([ 'Response'])[['Response']].count().plot(kind='bar').set_ylabel('Count')
df.groupby(['Response' ])[['Response']].count()

Univariate Analysis using Histogram to visualize the continous variables in the dataset. Below is an example of variable "BMI"

In [ ]:
# Univariate analysis- Histogram. Continous Variables: Product_Info_4, Ins_Age, Ht, Wt, BMI, Employment_Info_1, Employment_Info_4, 
# Employment_Info_6, Insurance_History_5, Family_Hist_2, Family_Hist_3, Family_Hist_4, Family_Hist_5
ax = df['BMI'].hist()
ax.set_ylabel('Counts')
ax.set_xlabel('BMI')
ax.set_title('BMI')
pd.DataFrame(df['BMI'].describe())

Bivariate Analysis: Effect of Age on Response (dependent variable). This analysis is used to check how age affects each of the responses.

In [ ]:
#Bivariate Analysis: Response v. Ins_Age
df.boxplot(column=['Ins_Age'],by = ['Response'])#.set_yscale('log')
pd.DataFrame(df.groupby( ['Response'])['Ins_Age'].describe().loc[:,['mean','std']])

Bivariate Analysis: Effect of BMI on Response. Here we see BMI clearly show a trend amongst the Responses. Notice Response-8 has a lower BMI 

In [ ]:
#Bivariate Analysis: Response v. BMI
df.boxplot(column=['BMI'], by = ['Response'])#.set_yscale('log')
pd.DataFrame(df.groupby( ['Response'])['BMI'].describe().loc[:,['mean','std']])

This is an example to check the effect of outliers on the Response. If we use 5th- 95th percentile, the outliers can be significantly reduced. However, the effect on Response is not that different. Hence, we keep all the data for analysis.

In [ ]:
# Choose percentile to miinimize the outliers
P = np.percentile(df['BMI'], [5, 95])
df_BMI = df[(df['BMI'] > P[0]) & (df['BMI'] < P[1])]
df_BMI.boxplot(column=['BMI'], by = ['Response'])#.set_yscale('log')
pd.DataFrame(df_BMI.groupby( ['Response'])['BMI'].describe().loc[:,['mean','std']])

Example of catogorical variable effect on Response. Each response is grouped by the Response percentage on the catogorical values

In [ ]:
#Response percentages for the catogorical variables - InsuredInfo_1

res =df.groupby(['InsuredInfo_1','Response'])[['Response']].sum()
Res_percent = res.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
print(Res_percent)
Res_percent.unstack().plot(kind='bar')
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

Another example of the effect of catogorical variable on the the Response grouped by percentage response

In [ ]:
# Response percentages for the catogorical variables -
res =df.groupby(['Medical_Keyword_20','Response'])[['Response']].sum()
Res_percent = res.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
print(Res_percent)
Res_percent.unstack().plot(kind='bar')
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

Correlation coefficants between independent variables to check for dependencies. Use "Pearson" method on continous variables and "Spearman" method on catogorical variables.

In [ ]:
#Correlation Coefficiants between independent variables
df[['Ins_Age','BMI','Ht', 'Wt']].corr(method='pearson')

In [ ]:
#Correlation Coefficiants between independent variables
df.loc[:,'Employment_Info_1':'Employment_Info_6'].corr(method='spearman')

In [ ]:
#Correlation Coefficiants between independent variables
df.loc[:,'Medical_History_1':'Medical_History_41'].corr(method='spearman')

Import modules for one way ANOVA table

In [39]:
# import modules for building ANOVA 
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

ANOVA table for F & P statistics

In [56]:
#ANOVA table
mod = ols('Medical_History_20 ~ Response', data = df).fit()
aov_table = sm.stats.anova_lm(mod, typ=1)
print(aov_table)

               df      sum_sq   mean_sq          F        PR(>F)
Response      1.0    5.230774  5.230774  357.19428  1.968745e-79
Residual  59379.0  869.549577  0.014644        NaN           NaN


F and P statistics and Hypothesis testing for H0(Null Hypothesis)

In [54]:
import numpy as np
from scipy import stats
f, p = stats.f_oneway(df['Medical_History_20'], df['Response'])
print('F value:', f)
print('P value:', p, '\n')

from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

mc = MultiComparison(df['Medical_History_20'], df['Response'])
result = mc.tukeyhsd()                        
print(result)                       
print(mc.groupsunique)                        

F value: 130870.37559455197
P value: 0.0 

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
  1      2    -0.0032  -0.0097 0.0033 False 
  1      3     0.0145   0.002  0.0269  True 
  1      4     0.0252   0.0145 0.036   True 
  1      5     0.0182   0.0114 0.025   True 
  1      6     0.0223   0.0165 0.028   True 
  1      7     0.0211   0.0149 0.0273  True 
  1      8     0.0246   0.0192 0.0299  True 
  2      3     0.0176   0.0053  0.03   True 
  2      4     0.0284   0.0177 0.0391  True 
  2      5     0.0214   0.0147 0.0281  True 
  2      6     0.0254   0.0197 0.0311  True 
  2      7     0.0243   0.0182 0.0304  True 
  2      8     0.0278   0.0225 0.033   True 
  3      4     0.0108  -0.0043 0.0258 False 
  3      5     0.0038  -0.0088 0.0163 False 
  3      6     0.0078  -0.0042 0.0198 False 
  3      7     0.0067  -0.0055 0.0189 False 
  3      8     0.0101  -0.0017 0.0219 False 
  4   